In [1]:
import matplotlib.pyplot as plt
from keras.layers import (
    Input, Dense, Conv2D, Flatten, Reshape, Dropout, 
    Activation, AveragePooling2D, BatchNormalization,
    Lambda, Concatenate
)
from keras.models import Model, load_model
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import image_dataset_from_directory
import numpy as np
from gc import collect

In [2]:
def WaveletTransform(img):
    low = img[:,::2,...] + img[:,1::2,...]
    low = (low[:,:,::2,...] + low[:,:,1::2,...]) / 4
    diag = img[:,::2,...] - img[:,1::2,...]
    diag = diag[:,:,::2,...] - diag[:,:,1::2,...]
    top = img[:,::2,...] - img[:,1::2,...]
    top = (top[:,:,::2] + top[:,:,1::2] ) / 2
    left = img[:,:,::2,...] - img[:,:,1::2,...]
    left = (left[:,::2] + left[:,1::2] ) / 2
    
    return low, diag, top, left


def conv_layer(_in, N):
    conv1 = Conv2D(N, kernel_size=(3,3), padding='same')(_in)
    norm1 = BatchNormalization()(conv1)
    relu1 = Activation('relu')(norm1)
    conv2 = Conv2D(N, kernel_size=(3,3), strides=(2,2), padding='same')(relu1)
    norm2 = BatchNormalization()(conv2)
    relu2 = Activation('relu')(norm2)
    return relu2


def build_model(input_shape = (256, 256, 3), num_classes=50):
    _input = BatchNormalization()(Input(input_shape))
    
    low1, diag1, top1, left1 = Lambda(WaveletTransform, name='wavelet_1')(_input)
    low2, diag2, top2, left2 = Lambda(WaveletTransform, name='wavelet_2')(low1)
    low3, diag3, top3, left3 = Lambda(WaveletTransform, name='wavelet_3')(low2)
    low4, diag4, top4, left4 = Lambda(WaveletTransform, name='wavelet_4')(low3)
    
    k1 = Concatenate()([conv_layer(_input,64), low1, diag1, top1, left1])
    k2 = Concatenate()([conv_layer(k1, 128), low2, diag2, top2, left2])
    k3 = Concatenate()([conv_layer(k2, 256), low3, diag3, top3, left3])
    k4 = Concatenate()([conv_layer(k3, 512), low4, diag4, top4, left4])
    
    avg_pool = AveragePooling2D(pool_size=(7,7), strides=1, padding='same')(k4)
    flat = Flatten()(avg_pool)
    output = Dense(num_classes, activation='softmax',name='fc')(flat)
    model = Model(inputs=_input, outputs=output)
    return model

In [3]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9817387720522389579
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14417788928
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17773940642470360236
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]
2.9.2
Num GPUs Available:  1
True


In [4]:
DO_KTH = False
DO_DTD = True

In [5]:
if DO_KTH:
  full_dataset = image_dataset_from_directory('./KTH-TIPS2-b', label_mode='categorical')
  print(full_dataset.class_names)
  DATASET_SIZE = 4752
  train_size = int(0.7 * DATASET_SIZE / 32)
  val_size = int(0.15 * DATASET_SIZE / 32)
  test_size = int(0.15 * DATASET_SIZE / 32)

  full_dataset = full_dataset.shuffle(10_000)
  train_dataset = full_dataset.take(train_size)
  test_dataset = full_dataset.skip(train_size)
  val_dataset = test_dataset.skip(test_size)
  test_dataset = test_dataset.take(test_size)
  # del full_dataset
  n = next(train_dataset.as_numpy_iterator())
  shape, num_classes = n[0].shape[1:], n[1].shape[-1]
  model = build_model(input_shape=shape, num_classes=num_classes)

  print(model.summary())

  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  !rm -rf ./checkpoints/*

  collect()

  checkpoint = ModelCheckpoint('./checkpoints/', monitor='accuracy', save_best_only=True)
  model.fit(train_dataset, validation_data=val_dataset, epochs=5, callbacks=[checkpoint])
  model = load_model('./checkpoints/')
  score = model.evaluate(test_dataset)
  N = sum([_[1].shape[1] for _ in test_dataset.as_numpy_iterator()])
  print(score)
  print(1/np.sqrt(N))

In [6]:
# !cp -r ./drive/MyDrive/dtd .

In [7]:
if DO_DTD:
  full_dataset = image_dataset_from_directory('./dtd/images/', label_mode='categorical')
  print(full_dataset.class_names)
  DATASET_SIZE = 5640
  train_size = int(0.7 * DATASET_SIZE / 32)
  val_size = int(0.15 * DATASET_SIZE / 32)
  test_size = int(0.15 * DATASET_SIZE / 32)

  full_dataset = full_dataset.shuffle(10_000)
  train_dataset = full_dataset.take(train_size)
  test_dataset = full_dataset.skip(train_size)
  val_dataset = test_dataset.skip(test_size)
  test_dataset = test_dataset.take(test_size)
  # del full_dataset
  shape, num_classes = full_dataset.element_spec
  shape, num_classes = tuple(shape.shape[1:]), int(num_classes.shape[1])
  model = build_model(input_shape=shape, num_classes=num_classes)

  print(model.summary())

  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  collect()

  checkpoint = ModelCheckpoint('./checkpoints/', monitor='accuracy', save_best_only=True)
  model.fit(train_dataset, validation_data=val_dataset, epochs=100, callbacks=[checkpoint])
  model = load_model('./checkpoints/')
  score = model.evaluate(test_dataset)
  N = sum([_[1].shape[1] for _ in test_dataset.as_numpy_iterator()])
  print(score)
  print(1/np.sqrt(N))

Found 5640 files belonging to 47 classes.
['banded', 'blotchy', 'braided', 'bubbly', 'bumpy', 'chequered', 'cobwebbed', 'cracked', 'crosshatched', 'crystalline', 'dotted', 'fibrous', 'flecked', 'freckled', 'frilly', 'gauzy', 'grid', 'grooved', 'honeycombed', 'interlaced', 'knitted', 'lacelike', 'lined', 'marbled', 'matted', 'meshed', 'paisley', 'perforated', 'pitted', 'pleated', 'polka-dotted', 'porous', 'potholed', 'scaly', 'smeared', 'spiralled', 'sprinkled', 'stained', 'stratified', 'striped', 'studded', 'swirly', 'veined', 'waffled', 'woven', 'wrinkled', 'zigzagged']
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
           

123/123 [==============================] - 108s 629ms/step - loss: 173.7665 - accuracy: 0.0284 - val_loss: 949.0280 - val_accuracy: 0.0190
Epoch 2/100
123/123 [==============================] - ETA: 0s - loss: 86.6893 - accuracy: 0.0465

123/123 [==============================] - 89s 613ms/step - loss: 86.6893 - accuracy: 0.0465 - val_loss: 114.3022 - val_accuracy: 0.0665
Epoch 3/100
123/123 [==============================] - ETA: 0s - loss: 88.9175 - accuracy: 0.0590

123/123 [==============================] - 87s 610ms/step - loss: 88.9175 - accuracy: 0.0590 - val_loss: 71.7547 - val_accuracy: 0.0770
Epoch 4/100
123/123 [==============================] - ETA: 0s - loss: 75.9447 - accuracy: 0.0780

123/123 [==============================] - 87s 610ms/step - loss: 75.9447 - accuracy: 0.0780 - val_loss: 76.0407 - val_accuracy: 0.1038
Epoch 5/100
123/123 [==============================] - ETA: 0s - loss: 75.7295 - accuracy: 0.0831

...

123/123 [==============================] - 87s 609ms/step - loss: 18.2626 - accuracy: 0.7620 - val_loss: 16.9586 - val_accuracy: 0.7578
Epoch 84/100
123/123 [==============================] - 82s 568ms/step - loss: 22.7892 - accuracy: 0.7416 - val_loss: 27.6887 - val_accuracy: 0.7018
Epoch 85/100
123/123 [==============================] - 82s 567ms/step - loss: 20.3506 - accuracy: 0.7464 - val_loss: 24.9818 - val_accuracy: 0.7422
Epoch 86/100
123/123 [==============================] - 82s 568ms/step - loss: 21.6195 - accuracy: 0.7423 - val_loss: 16.3250 - val_accuracy: 0.7824
Epoch 87/100
123/123 [==============================] - 82s 568ms/step - loss: 21.3924 - accuracy: 0.7485 - val_loss: 41.5073 - val_accuracy: 0.6842
Epoch 88/100
123/123 [==============================] - ETA: 0s - loss: 17.6666 - accuracy: 0.7671

123/123 [==============================] - 86s 605ms/step - loss: 17.6666 - accuracy: 0.7671 - val_loss: 28.1514 - val_accuracy: 0.7041
Epoch 89/100
123/123 [==============================] - 82s 568ms/step - loss: 19.5270 - accuracy: 0.7592 - val_loss: 20.2942 - val_accuracy: 0.7254
Epoch 90/100
123/123 [==============================] - 82s 568ms/step - loss: 18.4121 - accuracy: 0.7610 - val_loss: 29.6010 - val_accuracy: 0.7221
Epoch 91/100
123/123 [==============================] - ETA: 0s - loss: 18.3320 - accuracy: 0.7716

123/123 [==============================] - 87s 610ms/step - loss: 18.3320 - accuracy: 0.7716 - val_loss: 24.1408 - val_accuracy: 0.7634
Epoch 92/100
123/123 [==============================] - ETA: 0s - loss: 18.2421 - accuracy: 0.7840

123/123 [==============================] - 87s 608ms/step - loss: 18.2421 - accuracy: 0.7840 - val_loss: 33.4701 - val_accuracy: 0.7042
Epoch 93/100
123/123 [==============================] - 82s 567ms/step - loss: 19.9788 - accuracy: 0.7758 - val_loss: 34.7260 - val_accuracy: 0.6953
Epoch 94/100
123/123 [==============================] - 82s 569ms/step - loss: 17.9031 - accuracy: 0.7745 - val_loss: 19.9118 - val_accuracy: 0.7690
Epoch 95/100
123/123 [==============================] - ETA: 0s - loss: 13.1647 - accuracy: 0.8177

123/123 [==============================] - 87s 607ms/step - loss: 13.1647 - accuracy: 0.8177 - val_loss: 10.7892 - val_accuracy: 0.8154
Epoch 96/100
123/123 [==============================] - 82s 567ms/step - loss: 17.2012 - accuracy: 0.7965 - val_loss: 13.1590 - val_accuracy: 0.8237
Epoch 97/100
123/123 [==============================] - 82s 568ms/step - loss: 19.6873 - accuracy: 0.7779 - val_loss: 35.2803 - val_accuracy: 0.7299
Epoch 98/100
123/123 [==============================] - 82s 568ms/step - loss: 19.0420 - accuracy: 0.7802 - val_loss: 23.3108 - val_accuracy: 0.7746
Epoch 99/100
123/123 [==============================] - 82s 567ms/step - loss: 16.6042 - accuracy: 0.7955 - val_loss: 27.1714 - val_accuracy: 0.7634
Epoch 100/100
26/26 [==============================] - 18s 126ms/step - loss: 13.7569 - accuracy: 0.7993
[13.756921768188477, 0.7992788553237915]
0.028606478384531155
